In [2]:
from langchain_core.messages import AIMessage, ToolCall
from typing import Callable


def build_pre_model_hook(
    original_hook: Callable, *, force_tool_name: str | None, query: str
) -> Callable:
    def wrapped(state):
        if force_tool_name:
            # 강제 tool call 메시지 생성
            return {
                "messages": [
                    AIMessage(
                        tool_calls=[ToolCall(name=force_tool_name, args={"query": query})],
                        content=None,
                    )
                ]
            }
        return original_hook(state)

In [5]:
from typing import Any, Optional, Union
from enum import Enum, auto, StrEnum
from pydantic import BaseModel, Field

class OpenAIChatModelNamesEnum(StrEnum):
    GPT4O = auto()
    GPT4O_MINI = auto()
    GPT4O_MINI_FREE = (
        auto()
    )  # GPT4O_MINI랑 동일하게 gpt-4o-mini를 사용하는데, 크레딧 과금이 안 되는 무료 모델
    GPT_4_1 = auto()
    GPT_4_1_MINI = auto()
    GPT_4_1_NANO = auto()


ChatModelNamesEnum = Union[
    OpenAIChatModelNamesEnum,
]

class ChatModelConfig(BaseModel):
    model_name: str
    """모델 제공자가 정한 모델명"""
    temperature: float | None = 0.7
    context_length: int
    """컨텍스트 길이. 모델 제공자가 정한 값과 일치해야 합니다. 조절 불가능한 값입니다."""
    max_tokens: int
    (
        """최대 생성 토큰 수. 모델 제공자가 정한 값보다 작거나 같아야 합니다. """
        """(context_length - max_tokens)가 최대 입력 토큰 수 이므로, 너무 크게 잡아선 안 됩니다."""
    )
    vision_capability: bool
    """이미지 입력을 지원하는지 여부. 모델 제공자가 공개한 값과 일치해야 합니다."""
    knowledge_cutoff: str
    """모델의 지식 컷오프 날짜. 모델 제공자가 공개한 값과 일치해야 합니다."""
    default_kwargs: dict[str, Any] = Field(default_factory=dict)
    """모델 초기화 시 전달할 기본 키워드 인자들."""
    default_metadata: dict[str, Any] = Field(default_factory=dict)
    """모델 초기화 시 전달할 기본 langchain 메타데이터."""


CHAT_MODEL_CONFIG_MAP = {
    OpenAIChatModelNamesEnum.GPT4O_MINI: ChatModelConfig(
        model_name="gpt-4o-mini",
        context_length=128000,
        max_tokens=16384,
        vision_capability=True,
        knowledge_cutoff="October 2023",
    )
}

def get_max_input_tokens(model_name: ChatModelNamesEnum):
    """
    주어진 모델의 최대 입력 토큰 수를 반환합니다.
    """
    config = CHAT_MODEL_CONFIG_MAP.get(model_name)
    if config is None:
        raise ValueError(f"알 수 없는 모델명: {model_name}")

    # 최대 입력 토큰 수는 컨텍스트 길이에서 최대 생성 토큰 수를 뺀 값입니다.
    max_input_tokens = config.context_length - config.max_tokens

    return max_input_tokens

In [7]:
from langchain_core.messages import BaseMessage, HumanMessage, trim_messages

llm_name = OpenAIChatModelNamesEnum.GPT4O_MINI

max_input_tokens = get_max_input_tokens(llm_name)

print(max_input_tokens)
# 시스템 프롬프트, tool을 고려해서 10% 여유를 둠
max_input_tokens = int(max_input_tokens * 0.9)
message_trimmer = trim_messages(
    max_tokens=max_input_tokens,
    token_counter=approximate_token_counter,
    start_on=HumanMessage,
)

111616


NameError: name 'trim_messages' is not defined

In [ ]:
pre_model_hook = (
        (lambda x: x["messages"])
        | message_trimmer
        | convert_to_fn
        | (lambda x: {"llm_input_messages": x})
    )

In [3]:
custom_pre_model_hook = build_pre_model_hook(
    original_hook=pre_model_hook,
    force_tool_name="web_search" if web_search_option == "Y" else None,
    query=query,
)

NameError: name 'pre_model_hook' is not defined

In [11]:
from langchain_core.messages import trim_messages, HumanMessage, AIMessage, SystemMessage
from langchain_core.messages.utils import count_tokens_approximately


messages = [
    SystemMessage("너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해."),
    HumanMessage("최근에 발표된 LangChain 관련 주요 업데이트를 알려줘."),
    AIMessage("LangChain에서는 최근에 'LangGraph' 기능이 도입되었으며, 에이전트 워크플로우를 그래프 형태로 구성할 수 있게 되었습니다. (출처: LangChain 공식 블로그)"),
    HumanMessage("LangChain의 공동 창업자인 Harrison은 현재 무슨 프로젝트를 주도하고 있어?"),
    AIMessage("Harrison Chase는 LangChain의 CEO로, 최근에는 LangGraph 기반의 멀티에이전트 프레임워크 개발을 리드하고 있습니다. GitHub에서 관련 리포지토리를 운영 중입니다."),
    HumanMessage("웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘."),
]


trimmed = trim_messages(
    messages,
    max_tokens=45,
    strategy = "last",
    token_counter = count_tokens_approximately,
    start_on="human",
    end_on = ("human", "tool"),
    include_system=True,
    allow_partial=False,
)

In [12]:
trimmed

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [42]:
trimmed = trim_messages(
    messages,
    max_tokens=5,
    strategy="last",
    token_counter=len, #각 메시지를 1토큰으로 취급
    start_on = "human",
    end_on = ("human", "tool"),
    include_system=True,
)

trimmed

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='LangChain의 공동 창업자인 Harrison은 현재 무슨 프로젝트를 주도하고 있어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Harrison Chase는 LangChain의 CEO로, 최근에는 LangGraph 기반의 멀티에이전트 프레임워크 개발을 리드하고 있습니다. GitHub에서 관련 리포지토리를 운영 중입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [43]:
trimmed = trim_messages(
    messages,
    max_tokens=1,
    strategy="last",
    token_counter=len, #각 메시지를 1토큰으로 취급
    start_on = "human",
    end_on = ("human", "tool"),
    include_system=True,
)

trimmed

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={})]

In [40]:

sample_messages = [
    AIMessage("LangChain에서는 최근에 'LangGraph' 기능이 도입되었으며, 에이전트 워크플로우를 그래프 형태로 구성할 수 있게 되었습니다. (출처: LangChain 공식 블로그)"),
    HumanMessage("LangChain의 공동 창업자인 Harrison은 현재 무슨 프로젝트를 주도하고 있어?"),
    AIMessage("Harrison Chase는 LangChain의 CEO로, 최근에는 LangGraph 기반의 멀티에이전트 프레임워크 개발을 리드하고 있습니다. GitHub에서 관련 리포지토리를 운영 중입니다."),
    HumanMessage("웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘."),
]


trimmed = trim_messages(
    sample_messages,
    max_tokens=2,
    strategy="last",
    token_counter=len, #각 메시지를 1토큰으로 취급
    start_on = "human",
    end_on = ("human", "tool"),
    include_system=True,
)

trimmed 

[HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [44]:
trimmed = trim_messages(
    messages,
    max_tokens=2,
    strategy="last",
    token_counter=len, #각 메시지를 1토큰으로 취급
    start_on = "human",
    end_on = ("human", "tool"),
    include_system=True,
)

trimmed 

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [48]:
trimmed = trim_messages(
    messages,
    max_tokens=45,
    strategy="first",
    token_counter=count_tokens_approximately,
)

trimmed 

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='최근에 발표된 LangChain 관련 주요 업데이트를 알려줘.', additional_kwargs={}, response_metadata={})]

In [53]:
trim_messages(messages, max_tokens=2, include_system=False, token_counter=len)

[AIMessage(content='Harrison Chase는 LangChain의 CEO로, 최근에는 LangGraph 기반의 멀티에이전트 프레임워크 개발을 리드하고 있습니다. GitHub에서 관련 리포지토리를 운영 중입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [54]:
trim_messages(messages, max_tokens=2, include_system=True, token_counter=len)

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [34]:
trimmed = trim_messages(
    messages,
    max_tokens=4,
    strategy="last",
    token_counter=len, #각 메시지를 1토큰으로 취급
    start_on = "human",
    end_on = ("human", "tool"),
    include_system=True,
)

trimmed

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='LangChain의 공동 창업자인 Harrison은 현재 무슨 프로젝트를 주도하고 있어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Harrison Chase는 LangChain의 CEO로, 최근에는 LangGraph 기반의 멀티에이전트 프레임워크 개발을 리드하고 있습니다. GitHub에서 관련 리포지토리를 운영 중입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [59]:
from langchain_openai import ChatOpenAI

trim_messages(
    messages,
    max_tokens=100,
    strategy="first",
    token_counter=ChatOpenAI(model='gpt-4o-mini')
)

[SystemMessage(content='너는 웹 검색 도구를 활용해 정보를 조사하고 요약해주는 인턴 AI야. 답변할 때는 출처나 맥락을 명확히 전달하고, 필요한 경우 검색 도구를 사용해서 가장 최신 정보를 찾아야 해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='최근에 발표된 LangChain 관련 주요 업데이트를 알려줘.', additional_kwargs={}, response_metadata={})]

In [63]:
from typing import List
from langchain_core.messages import BaseMessage
import tiktoken

def tiktoken_counter(messages: List[BaseMessage]) -> int:
    enc = tiktoken.get_encoding('o200k_base')
    return sum(len(enc.encode(msg.content)) +3 for msg in messages)


trim_messages(messages, token_counter=tiktoken_counter, max_tokens=45)

[HumanMessage(content='웹 기반 에이전트 개발을 위한 대표적인 오픈소스 예시 알려줘.', additional_kwargs={}, response_metadata={})]

In [65]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    start_on="human",
    include_system=True,
)

chain = trimmer | llm
response = chain.invoke(messages)
response

AIMessage(content='저는 웹 검색 도구를 사용할 수 없지만, 최신 정보에 대한 질문에 대해 알고 있는 내용을 바탕으로 답변을 드릴 수 있습니다. 질문이 있으면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 61, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrHQxZ5dXzQED97niPn6U4Yh4wcsD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f06ad4aa-6c56-4204-99cc-81969afe2546-0', usage_metadata={'input_tokens': 61, 'output_tokens': 41, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [66]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

history = InMemoryChatMessageHistory(messages=messages[:-1])

def get_history(session_id):
    return history if session_id == "1" else InMemoryChatMessageHistory()


chain_with_history = RunnableWithMessageHistory(chain, get_history)

response = chain_with_history.invoke(
    [HumanMessage("최근 서울 날씨 알려줘")],
    config= {"configurable" : {"session_id" : "1"}},
)

In [67]:
response

AIMessage(content='알겠습니다! 필요한 정보를 조사하고 요약하여 드리겠습니다. 질문이나 요청이 있으시면 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 61, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrHU3andVDODdlu005QoEVb5HDXtu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e44f76e3-119a-4ce0-b908-98257ae54be7-0', usage_metadata={'input_tokens': 61, 'output_tokens': 24, 'total_tokens': 85, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})